In [1]:
import sqlite3
import pandas as pd
import os
import matplotlib

In [115]:
# Connect to the database
db_connection = sqlite3.connect('mydatabase.db')
query = "SELECT * FROM box_scores"
query2 = "SELECT * FROM fixture_information"
query3 = "SELECT * FROM test_fixtures"
query4 = "SELECT * FROM test_fixtures_actuals"
df_box_scores = pd.read_sql_query(query, db_connection)
df_fixture_info = pd.read_sql_query(query2, db_connection)
df_test_fixtures = pd.read_sql_query(query3, db_connection)
df_test_fixtures_actuals = pd.read_sql_query(query4, db_connection)
db_connection.close()


In [10]:
#only missing of NA values in the dataframes is the handicap column in the df_test_fixtures_actuals
df_box_scores
df_fixture_info
df_test_fixtures
df_test_fixtures_actuals

,FixtureKey,Team,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,IsWinner,TeamHandicap
0,ID ST v E WASH 25-Feb-2023,2,9,19,11,31,12,15,8,25,15,3,4,19,23,63,0,-6.0
1,ID ST v E WASH 25-Feb-2023,1,13,27,10,28,15,20,9,24,15,7,3,11,19,71,1,6.0
2,WEB ST v IDAHO 25-Feb-2023,2,7,24,9,23,12,19,4,28,10,4,2,13,17,53,0,8.0
3,WEB ST v IDAHO 25-Feb-2023,1,16,35,9,23,8,10,6,32,11,8,1,8,16,67,1,-8.0
4,MONT v SAC ST 25-Feb-2023,1,18,30,8,23,14,20,7,16,12,7,2,7,18,74,1,-7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,NC CEN v HOWARD 25-Feb-2023,1,21,41,4,17,14,17,14,26,11,5,8,15,14,68,1,NaN
256,JACKST v TX SOU 25-Feb-2023,1,22,41,4,10,15,18,6,19,8,9,2,9,10,71,1,-1.5
257,JACKST v TX SOU 25-Feb-2023,2,24,46,5,14,6,6,9,20,8,6,2,12,16,69,0,1.5
258,BUCKNL v LAFAYE 25-Feb-2023,1,17,26,9,27,14,19,6,27,20,7,4,9,13,75,1,NaN


In [12]:
df_test_fixtures_actuals.isna()

,FixtureKey,Team,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,IsWinner,TeamHandicap
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
256,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
257,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
258,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [50]:
df_box_scores.describe()

,FixtureKey,Team,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF
0,LIPSCO v A PEAY 14-Jan-2023,2,18,41,7,21,15,16,12,19,13,7,4,12,21
1,LIPSCO v A PEAY 14-Jan-2023,1,16,29,10,22,24,29,7,25,17,5,2,13,18
2,QUENNC v A PEAY 29-Dec-2022,2,20,44,8,18,13,15,9,19,18,6,2,10,26
3,QUENNC v A PEAY 29-Dec-2022,1,18,32,7,19,24,36,11,25,14,4,2,13,18
4,FGCU v A PEAY 24-Feb-2023,2,20,29,8,33,7,11,8,23,19,6,1,10,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13317,YSU v SIUE 20-Nov-2021,1,18,31,5,17,15,23,7,23,9,9,3,18,19
13318,YSU v OAK 09-Feb-2022,2,18,32,5,23,20,25,10,24,13,7,1,14,16
13319,YSU v OAK 09-Feb-2022,1,21,35,9,33,9,11,12,25,20,4,2,9,21
13320,DET v YSU 12-Jan-2023,1,26,44,6,23,9,12,11,26,12,3,2,5,16


In [1]:
df_box_scores_2 = pd.DataFrame()
df_box_scores_2['HomeTeam'] = df_box_scores['FixtureKey'].apply(lambda x:x.split(" v ")[0])
df_box_scores_2['AwayTeam'] = df_box_scores['FixtureKey'].apply(lambda x:x.split(" v ")[1])
df_box_scores_2['AwayTeam'] = df_box_scores_2['AwayTeam'].apply(lambda x:x.replace(x[-11:],""))
df_box_scores_2['Dates'] = df_box_scores['FixtureKey'].apply(lambda x: x.split(x[:-11])[1])
df_box_scores_2 = df_box_scores_2.set_index('Dates', drop=True)
for i in np.arange(2,len(df_box_scores.columns),1):
    df_box_scores_2[df_box_scores.columns[i]] = df_box_scores[df_box_scores.columns[i]].tolist()
df_box_scores_2['X2PM'].groupby(df_box_scores_2['HomeTeam']).mean()
df_box_scores_2['X2PM'].groupby(df_box_scores_2['AwayTeam']).mean()

NameError: name 'pd' is not defined